In [1]:
import ee
import geemap

In [2]:
ee.Initialize()

In [3]:
aoi = ee.FeatureCollection("projects/ee-tomarishan1988/assets/fpo_1")

In [67]:
START_DATE = '2023-01-15'
END_DATE = '2023-03-25'
CLOUD_FILTER = 1
CLD_PRB_THRESH = 50
NIR_DRK_THRESH = 0.15
#CLD_PRJ_DIST = 10 
BUFFER = 50

In [20]:
s2 = ee.ImageCollection('COPERNICUS/S2') \
    .filterBounds(aoi) \
    .filterDate(START_DATE, END_DATE) \
    .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE',1))

In [84]:
s2 = ee.ImageCollection('COPERNICUS/S2') \
    .filter(ee.Filter.equals("MGRS_TILE", "44QKF")) \
    .filterDate(START_DATE, END_DATE)
    #.filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE',10))

In [6]:
c2 = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY') \
    .filterBounds(aoi) \
    .filterDate(START_DATE, END_DATE)

In [85]:
s2

Name,Description
B1,Aerosols
B2,Blue
B3,Green
B4,Red
B5,Red Edge 1
B6,Red Edge 2
B7,Red Edge 3
B8,NIR
B8A,Red Edge 4
B9,Water vapor


In [77]:
out_dir

'D:\\Images\\cloudfromgee'

In [11]:
geom = aoi.geometry()

In [12]:
out_dir = r"D:\Images\cloudfromgee"

In [ ]:
Map=geemap.Map()
Map

In [15]:
geemap.download_ee_image_collection(c2, out_dir, scale=10,region=geom)

Total number of images: 4



20230317T051651_20230317T052835_T44QKF.tif: |          | 0.00/1.21M (raw) [  0.0%] in 00:00 (eta:     ?)

20230319T050659_20230319T052158_T44QKF.tif: |          | 0.00/1.21M (raw) [  0.0%] in 00:00 (eta:     ?)

20230322T051659_20230322T053209_T44QKF.tif: |          | 0.00/1.21M (raw) [  0.0%] in 00:00 (eta:     ?)

20230324T050651_20230324T051534_T44QKF.tif: |          | 0.00/1.21M (raw) [  0.0%] in 00:00 (eta:     ?)

In [81]:
geemap.download_ee_image_collection(s2, out_dir, scale=10)

Exception: Error downloading image collection: Collection.filter: Expression parse error at character 15:
'MGRS_TILE == 44QKF'
                ^.

In [22]:
d2 = ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
    'primary': s2,
    'secondary': c2,
    'condition': ee.Filter.equals(**{
        'leftField':'system:index',
        'rightField': 'system:index'
    })
}))

In [24]:
geemap.download_ee_image_collection(d2, out_dir, scale=10,region=geom)

Total number of images: 1



20230324T050651_20230324T051534_T44QKF.tif: |          | 0.00/77.7M (raw) [  0.0%] in 00:00 (eta:     ?)

In [25]:
def add_cloud_bands(img):
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))

In [26]:
e2 = d2.map(add_cloud_bands)

In [30]:
e2

Name,Description
B1,Aerosols
B2,Blue
B3,Green
B4,Red
B5,Red Edge 1
B6,Red Edge 2
B7,Red Edge 3
B8,NIR
B8A,Red Edge 4
B9,Water vapor


In [29]:
geemap.download_ee_image_collection(e2, out_dir, scale=10,region=geom)

Total number of images: 1



20230324T050651_20230324T051534_T44QKF.tif: |          | 0.00/87.4M (raw) [  0.0%] in 00:00 (eta:     ?)

In [59]:
def add_shadow_bands(img):
    # Identify water pixels from the SCL band.
    #ot_water = img.select('SCL').neq(6)

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    #dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));
    #print(shadow_azimuth)
    CLD_PRJ_DIST = 5
    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 10})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))
    #return img.addBands(ee.Image([cld_proj]))

In [60]:
f2 = e2.map(add_shadow_bands)

In [61]:
geemap.download_ee_image_collection(f2, out_dir, scale=10,region=geom)

Total number of images: 1



20230324T050651_20230324T051534_T44QKF.tif: |          | 0.00/204M (raw) [  0.0%] in 00:00 (eta:     ?)